In [1]:
from datetime import datetime
import json
import os
from os.path import join as pjoin
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go


DATA_DIR = './data'

probes_mean_time = []
config_dates = []

for filename in os.listdir(DATA_DIR):

    filename = pjoin(DATA_DIR, filename)
    with open(filename) as data_file:
        data_string = data_file.read()
        try:
            df = json.loads(data_string)
            probes_mean_time.append(df['Probes'][0]['Mean'])
            config_dates.append(df['ITK_MANUAL_BUILD_INFO']['GIT_CONFIG_DATE'])
        except ValueError:
            print(repr(data_string))

data = [go.Scatter(
          x=config_dates,
          y=probes_mean_time)]
py.sign_in('name', 'API_key')
py.iplot(data)
